# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_data_path = '../WeatherPy/output_data/cities.csv'
cities_df = pd.read_csv(cities_data_path)
print(cities_df.count())
cities_df.head()

City          547
Cloudiness    547
Country       541
Date          547
Humidity      547
Lat           547
Lng           547
Max Temp      547
Wind Speed    547
dtype: int64


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Elizabeth,90,ZA,1585709292,93,-33.92,25.57,60.800,10.289924
1,Jiangkou,75,CN,1585709293,82,25.49,119.20,60.800,13.421640
2,Gushikawa,90,JP,1585709293,100,26.35,127.87,68.000,14.987498
3,Mataura,10,NZ,1585709293,69,-46.19,168.86,66.002,2.997500
4,Kapaa,40,US,1585709293,69,22.08,-159.32,78.800,14.987498


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key) # Fill in with your API key

locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']

In [18]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50,
                                 point_radius = 2, opacity=0.5))
embed_minimal_html('./output_data/vacation_heatmap.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
vac_cities_df = cities_df.copy()
vac_cities_df = vac_cities_df.loc[(vac_cities_df['Max Temp'] > 70) & 
                                  (vac_cities_df['Max Temp'] < 80) & 
                                  (vac_cities_df['Wind Speed'] < 10) &
                                  (vac_cities_df['Cloudiness'] == 0)]

vac_cities_df.count()

City          5
Cloudiness    5
Country       5
Date          5
Humidity      5
Lat           5
Lng           5
Max Temp      5
Wind Speed    5
dtype: int64

In [8]:
vac_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
243,Cockburn Town,0,TC,1585709350,72,21.46,-71.14,76.784,9.506995
298,Umluj,0,SA,1585709366,41,25.02,37.27,73.922,2.572481
436,Lakes Entrance,0,AU,1585709399,49,-37.88,147.98,73.004,1.990877
447,Tessalit,0,ML,1585709402,15,20.20,1.01,70.556,6.487126
476,Ormara,0,PK,1585709410,73,25.21,64.64,73.346,7.538488


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = vac_cities_df.copy()

In [11]:
for index, data in hotel_df.iterrows():
    print(index, data['Lat'], data['Lng'])
    latitude = data['Lat']
    longitude = data['Lng']
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    hotel_name = response['results'][0]['formatted_address']
    hotel_df.loc[index, 'Hotel Name'] = hotel_name
    print(hotel_name)
    

243 21.46 -71.14
Cockburn Town TKCA 1ZZ, Turks and Caicos Islands
298 25.02 37.27
Umluj Saudi Arabia
436 -37.88 147.98
Lakes Entrance VIC 3909, Australia
447 20.2 1.01
Achemelmel, Mali
476 25.21 64.64
Ormara, Gwadar, Balochistan, Pakistan


In [12]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
243,Cockburn Town,0,TC,1585709350,72,21.46,-71.14,76.784,9.506995,"Cockburn Town TKCA 1ZZ, Turks and Caicos Islands"
298,Umluj,0,SA,1585709366,41,25.02,37.27,73.922,2.572481,Umluj Saudi Arabia
436,Lakes Entrance,0,AU,1585709399,49,-37.88,147.98,73.004,1.990877,"Lakes Entrance VIC 3909, Australia"
447,Tessalit,0,ML,1585709402,15,20.20,1.01,70.556,6.487126,"Achemelmel, Mali"
476,Ormara,0,PK,1585709410,73,25.21,64.64,73.346,7.538488,"Ormara, Gwadar, Balochistan, Pakistan"


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50, point_radius = 2, opacity=1))
fig.add_layer(gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info))
embed_minimal_html('./output_data/vacation_hotels.html', views=[fig])
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))